<a href="https://colab.research.google.com/github/blankazselyke/Onlab/blob/main/%C3%96nlab_DPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install transformers
!pip3 install datasets
!pip3 install trl
!pip install -q -U bitsandbytes
!pip install accelerate
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 14.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-m

In [3]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer,TrainingArguments, AutoModelForQuestionAnswering
from trl import DPOTrainer, SFTTrainer
from transformers import BitsAndBytesConfig
from torch import bfloat16
import torch
from peft import LoraConfig,PeftModel, PeftConfig, get_peft_model, AutoPeftModelForCausalLM
import warnings
import os
warnings.filterwarnings("ignore")

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)
#Even though the weights are loaded in a 4-bit format,
#computations during forward and backward passes will be performed using bfloat16 data type,
#which is a 16-bit floating-point representation.
#bfloat16 strikes a balance between memory efficiency and computational precision.
model_name ="facebook/opt-350m"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
base_model.config.use_cache = False

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [5]:
base_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
      (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear4bit(in_features=1024, out_features=4096, bias=True)
          (

In [6]:
proba = '/content/train.csv'

In [7]:
def formatting_prompts_func(example):
  output_texts = []
  assert len(example["text"]) == len(example["label"])
  for i in range(len(example['text'])):
    text = f"### Input: ```{example['text'][i]}```\n ### Output: {example['label'][i]}"
    output_texts.append(text)
  return output_texts

In [8]:
import os
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [9]:
sft_dataset = load_dataset("gbharti/finance-alpaca", split="train")
sft_datset_new = sft_dataset.select(range(10000))
sft_splitted_data = sft_datset_new.train_test_split(test_size=0.3,seed=42, shuffle=True)

sft_splitted_data["train"] = sft_splitted_data["train"].remove_columns(["input", "text"])
sft_splitted_data["test"] = sft_splitted_data["test"].remove_columns(["input", "text"])

sft_splitted_data["train"] = sft_splitted_data["train"].rename_column("output", "label")
sft_splitted_data["train"] = sft_splitted_data["train"].rename_column("instruction", "text")

sft_splitted_data["test"]  = sft_splitted_data["test"].rename_column("output", "label")
sft_splitted_data["test"]  = sft_splitted_data["test"].rename_column("instruction", "text")

Generating train split:   0%|          | 0/68912 [00:00<?, ? examples/s]

In [10]:
sft_splitted_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3000
    })
})

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
# training the supervised model
"""
 max_step = epoch*(total_example/batch_size)
 logging_step = total_example/batch_size
 loggers = logging_step/epoch
"""
output_path = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/"
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["k_proj","v_proj","q_proj","out_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)
MAX_STEPS =int(2 * len(sft_splitted_data["train"]) / 4)
args = TrainingArguments(
        output_path,
        overwrite_output_dir=True, # This reduces the amt of disk space that gets used.
        fp16=True,  # fp16 training to allow larger batch sizes to be used
        evaluation_strategy = "steps",
        save_strategy = "steps",
        learning_rate=5e-5,
        warmup_steps=500,
        warmup_ratio=0.1,
        do_train=True,
        do_eval=True,
        logging_steps=500,
        eval_steps=500,
        save_total_limit=1,
        gradient_accumulation_steps=2,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        max_steps=MAX_STEPS,
        weight_decay=0.01,
        dataloader_num_workers=4,
        load_best_model_at_end=True
)

trainer = SFTTrainer(
    model=base_model,
    train_dataset=sft_splitted_data["train"],
    eval_dataset=sft_splitted_data["test"],
    tokenizer=tokenizer,
    peft_config=config,
    formatting_func=formatting_prompts_func,
    dataset_text_field="text",
    args=args,         # Trainer arguments
    max_seq_length=256,
)
trainer.train()

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss
500,3.943900,3.632558
1000,3.642100,3.555183
1500,3.548700,3.507126
2000,3.528900,3.469328
2500,3.470500,3.445009
3000,3.431200,3.428911
3500,3.427900,3.422465


TrainOutput(global_step=3500, training_loss=3.5704368373325894, metrics={'train_runtime': 2221.5313, 'train_samples_per_second': 12.604, 'train_steps_per_second': 1.575, 'total_flos': 1059394415493120.0, 'train_loss': 3.5704368373325894, 'epoch': 4.0})

In [12]:
output_path = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/"
adapter_path = os.path.join(output_path,"checkpoint-3500")

In [13]:
math_dataset = load_dataset("argilla/distilabel-math-preference-dpo", split="train")
def process_dataset(sample_data):
  return {
      "prompt": [f"Question: " + question + "\n\nAnswer: "
      for question in sample_data["instruction"]
      ],
      "chosen": sample_data["chosen_response"],
      "rejected": sample_data["rejected_response"]
  }

original_cols = math_dataset.column_names
math_dataset = math_dataset.map(process_dataset,batched=True,remove_columns=original_cols)
# let's divide the dataset
math_dataset_splits = math_dataset.train_test_split(test_size=0.2,seed=42,shuffle=True)
math_dataset_splits

Generating train split:   0%|          | 0/2418 [00:00<?, ? examples/s]

Map:   0%|          | 0/2418 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1934
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 484
    })
})

In [14]:
# now we can train the DPO model based on the dataset we create
model = AutoPeftModelForCausalLM.from_pretrained(
 adapter_path,
 low_cpu_mem_usage=True,
 torch_dtype=torch.bfloat16,
 load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(adapter_path,trust_remote_code=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [15]:
MAX_STEPS =int(5 * len(math_dataset_splits["train"]) / 16)

output_path_dpo = os.path.join(output_path, "dpo_models")
args = TrainingArguments(
        output_path_dpo,
        overwrite_output_dir=True, # This reduces the amt of disk space that gets used.
        fp16=True,  # fp16 training to allow larger batch sizes to be used
        evaluation_strategy = "steps",
        save_strategy = "steps",
        learning_rate=5e-5,
        warmup_steps=500,
        warmup_ratio=0.1,
        do_train=True,
        do_eval=True,
        logging_steps=500,
        eval_steps=500,
        save_total_limit=1,
        gradient_accumulation_steps=2,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        max_steps=MAX_STEPS,
        weight_decay=0.01,
        dataloader_num_workers=4,
        load_best_model_at_end=True
)

In [16]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["k_proj","v_proj","q_proj","out_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

dpo_trainer = DPOTrainer(
    model,
    args=args,
    beta=0.1,
    train_dataset=math_dataset_splits["train"],
    eval_dataset=math_dataset_splits["test"],
    tokenizer=tokenizer,
    peft_config=config,
    max_length=512,
)
dpo_trainer.train()

Map:   0%|          | 0/1934 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
500,0.849900,0.719666,0.577838,0.560588,0.516529,0.017249,-595.854553,-616.615479,-0.876452,-1.197252


TrainOutput(global_step=604, training_loss=0.8342637194702957, metrics={'train_runtime': 863.7814, 'train_samples_per_second': 2.797, 'train_steps_per_second': 0.699, 'total_flos': 0.0, 'train_loss': 0.8342637194702957, 'epoch': 1.249224405377456})

In [17]:
!zip -r /content/lib.zip /content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/checkpoint-3500

  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/checkpoint-3500/ (stored 0%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/checkpoint-3500/README.md (deflated 66%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/checkpoint-3500/adapter_model.safetensors (deflated 7%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/checkpoint-3500/adapter_config.json (deflated 52%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/checkpoint-3500/tokenizer_config.json (deflated 63%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/checkpoint-3500/special_tokens_map.json (deflated 74%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/checkpoint-3500/vocab.json (deflated 59%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/checkpoint-3500/merges.txt (deflated 53%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/c

In [18]:
!rm -rf /content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/dpo_models/checkpoint-11500

In [19]:
adapter_model = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/dpo_models/checkpoint-500/"
# tokenizer = AutoTokenizer.from_pretrained(adapter_model)
# model = AutoModelForCausalLM.from_pretrained(base_model)
# model.resize_token_embeddings(len(tokenizer))
# peftmodel = PeftModel.from_pretrained(model, adapter_model)

model = AutoPeftModelForCausalLM.from_pretrained(adapter_model, torch_dtype=torch.bfloat16)
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(adapter_model)

In [20]:
model.eval()
for i in range(len(math_dataset_splits["test"][:20])):
  prompt = math_dataset_splits["test"][i]["prompt"]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=100,
    repetition_penalty=1.3
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

</s>Question: Explain the concept of conditional probability and provide an example..
 Take a deep breath, think step by step, and give an accurate response

Answer: 
The concept of conditional probability is used in the following examples.

1) A conditional probability equation (CBOE):

2) A conditional probability equation (PIE):

3) A conditional probability equation (PAI):

4) A conditional probability equation (PCY):

5) A conditional probability equation (PHO):

6) A conditional probability equation (PSA):

7) A conditional probability equation (SPF):


****************************************************************************************************
</s>Question: Detail the steps involved in finding the solution to a linear equation with three variables..
 Take a deep breath, think step by step, and give an accurate response

Answer: _______________________________________________

I have been trying to solve a linear problem for several years. I am not sure what is wrong or h

In [21]:
for i in range(len(math_dataset_splits["test"][:20])):
  prompt = math_dataset_splits["test"][i]["prompt"]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.5,
    top_p=0.95,
    top_k=40,
    max_new_tokens=100,
    repetition_penalty=1.3
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

</s>Question: Explain the concept of conditional probability and provide an example..
 Take a deep breath, think step by step, and give an accurate response

Answer: 
The concept of conditional probability is used in the following examples.

1) A conditional probability equation (CBOE):

2) A conditional probability equation (PIE):

3) A conditional probability equation (PAI):

4) A conditional probability equation (PCY):

5) A conditional probability equation (PHO):

6) A conditional probability equation (PSA):

7) A conditional probability equation (SPF):


****************************************************************************************************
</s>Question: Detail the steps involved in finding the solution to a linear equation with three variables..
 Take a deep breath, think step by step, and give an accurate response

Answer: _______________________________________________

I have been trying to solve a linear problem for several years. I am not sure what is wrong or h

In [22]:
general_prompts = ["Hello! How are you today?", "What is 5 + 2?", "What is 5 * 2?", "Please tell me your favourite joke.", "Why is it impossible to divide by 0?", "Please give me a recipe for lunch that contains tomato."]

In [23]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=100,
    repetition_penalty=1.3
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

</s>Hello! How are you today?
I'm doing great. I have a lot of work to do, but it's all good :)  What about yourself?
Good morning! I hope your day is going well too :D</s>
****************************************************************************************************
</s>What is 5 + 2?
2
Calculate -1 + (0 - (-3)/(-6)).
-4/9
Evaluate 1*(5/(10/8) - 0).
7
((-12)/15)/((-18)/30)*11
22
What is the value of ((-20)/25)/((-16)/40)?
-4
What is 4/14 + 3/21?
-1/7
What is 690/
****************************************************************************************************
</s>What is 5 * 2?
5
Calculate the remainder when 6*(-2)/4 + -1.
0
Let x(z) = z**3 + 3*z**2 - 4*z - 1. Let y be (-6)/9*-7*12/8. What is the remainder when (y/(-10))/(18/30)*15 is divided by x(-3)?
11
Suppose 0 = 7*m, m - 8 = r -
****************************************************************************************************
</s>Please tell me your favourite joke.
I'm not sure, but I think it's the one about how you c

In [24]:
model.eval()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50265, 512)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features

In [25]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.35,
    top_p=0.45,
    top_k=20,
    max_new_tokens=50,
    repetition_penalty=0.75
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

</s>Hello! How are you today?
Hello! How are you?
Hello! How are you?
Hello! How are you?
Hello! How are you?
Hello! How are you?
Hello! How are you?
Hello! How are you?

****************************************************************************************************
</s>What is 5 + 2?

What is 5 + 2 + 2?

What is 2 + 2 + 2?

What is 2 + 2 + 2?

What is 2 + 2 + 2?

What is 2 + 2 + 2?
****************************************************************************************************
</s>What is 5 * 2?

What is 5 * 2?

What is 5 * 2?

What is 5 * 2?

What is 5 * 2?

What is 5 * 2?

What is 5 * 2?


****************************************************************************************************
</s>Please tell me your favourite joke.

Please tell me your favourite joke.

Please tell me your favourite joke.

Please tell me your favourite joke.

Please tell me your favourite joke.

Please tell me your favourite joke.

Please tell me
*************************************************

In [26]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.35,
    top_p=0.45,
    top_k=20,
    max_new_tokens=50,
    repetition_penalty=1.5
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

</s>Hello! How are you today?
I'm doing great. I have a lot of work to do, but it's all good :)  What about yourself?
Good morning! I hope your day is going well too :D
It was pretty awesome actually...and thank goodness
****************************************************************************************************
</s>What is 5 + 2?
2*(1 - 1)**3/4
Factor 3/(-6)*(-5)/15.
-(u / 6))/9
Let v = 0 + 4, 7 = d for z.
0
Supp
****************************************************************************************************
</s>What is 5 * 2?
5*2.8
-4 times -0.1
3/7
Calculate 0 + 1/(624/16).
10
Work out 3 (936 / 6) + (-12)/(-18)*
****************************************************************************************************
</s>Please tell me your favourite joke.
I'm not sure, but I think it's the one about how you can't get a job if you're gay or lesbian and that's why you have to be in school for so long before getting an education?   Edit: spelling
********************************

In [27]:
general_prompts = ["Hello! How are you today?", "Solve this mathematical problem: what is 5 + 2? For example, the answer to what is 5 + 1 is: 5 + 1 = 6. Think logically and step by step.", "What should I invest my money in?", "Please tell me your favourite joke.", "Why is it impossible to divide by 0?", "Please give me a recipe for lunch that contains tomato."]

In [28]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.35,
    top_p=0.45,
    top_k=20,
    max_new_tokens=50,
    repetition_penalty=1.5
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

</s>Hello! How are you today?
I'm doing great. I have a lot of work to do, but it's all good :)  What about yourself?
Good morning! I hope your day is going well too :D
It was pretty awesome actually...and thank goodness
****************************************************************************************************
</s>Solve this mathematical problem: what is 5 + 2? For example, the answer to what is 5 + 1 is: 5 + 1 = 6. Think logically and step by step.
I'm not sure I understand your question...  What's the difference between 5 & 3?
5 - 4 (or more) are different numbers in a number system that can be used as an algebraic equation for solving problems like this one
****************************************************************************************************
</s>What should I invest my money in?
The best way to get the most out of your investment portfolio is by investing it wisely. Investing a small amount each month can help you build up an asset class that will last for y

In [30]:
adapter_model = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft2/models/checkpoint-3500/"
# tokenizer = AutoTokenizer.from_pretrained(adapter_model)
# model = AutoModelForCausalLM.from_pretrained(base_model)
# model.resize_token_embeddings(len(tokenizer))
# peftmodel = PeftModel.from_pretrained(model, adapter_model)

model = AutoPeftModelForCausalLM.from_pretrained(adapter_model, torch_dtype=torch.bfloat16)
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(adapter_model)

In [31]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.35,
    top_p=0.45,
    top_k=20,
    max_new_tokens=50,
    repetition_penalty=1.5
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

</s>Hello! How are you today?
I'm good. I have a job and am getting my degree in finance, but not enough to be able afford it yet (and don't want the debt). What can I do with this money now that's available for me at an early
****************************************************************************************************
</s>Solve this mathematical problem: what is 5 + 2? For example, the answer to what is 5 + 1 is: 5 + 1 = 6. Think logically and step by step.
I'm not sure how to solve it but I think it's a good question! Thanks for your help :)  Edit - just realized that I have no idea where to start with this math thing... What are some examples of numbers in general or
****************************************************************************************************
</s>What should I invest my money in?
The stock market is a highly volatile business. It can be very unpredictable and difficult to predict, especially if you are young or have no experience with investing at all!

In [32]:
model.eval()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50265, 512)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features

In [33]:
model.eval()
for i in range(len(math_dataset_splits["test"][:20])):
  prompt = math_dataset_splits["test"][i]["prompt"]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=100,
    repetition_penalty=1.3
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

</s>Question: Explain the concept of conditional probability and provide an example..
 Take a deep breath, think step by step, and give an accurate response

Answer: _________

I have been asked to explain conditional probabilities. What is this? How do they work in the real world? (not sure if I should be asking)

What are conditional probabilities?

A conditional probability is a mathematical formula that describes how many times you can get something from one set of data points without affecting other sets or variables?

How does it relate to probability theory/mathematics?

The idea behind conditional probabilities comes down to the fact that
****************************************************************************************************
</s>Question: Detail the steps involved in finding the solution to a linear equation with three variables..
 Take a deep breath, think step by step, and give an accurate response

Answer: _____.

I have a problem that I can't solve using a line

In [34]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.35,
    top_p=0.45,
    top_k=20,
    max_new_tokens=50,
    repetition_penalty=1.5
    )
    outputs = base_model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

</s>Hello! How are you today? What is your current situation and what do I need to know about it before investing in a stock that has been on the decline for over 2 years now?!
I'm not sure how much money I have. Is there any way of finding out if
****************************************************************************************************
</s>Solve this mathematical problem: what is 5 + 2? For example, the answer to what is 5 + 1 is: 5 + 1 = 6. Think logically and step by step. What does 5 - 3 mean in this equation (and other equations)?
I'm not sure how to solve it... I have a calculator but can't figure out where to put my numbers! Help!! Thanks!!!
Answer below for the question "
****************************************************************************************************
</s>What should I invest my money in? What is the best way to save for retirement, and what are some good investments that can help me do so (and not be a total loser)?
I'm thinking of investing it as

In [35]:
model.eval()
for i in range(len(math_dataset_splits["test"][:20])):
  prompt = math_dataset_splits["test"][i]["prompt"]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=100,
    repetition_penalty=1.3
    )
    outputs = base_model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

</s>Question: Explain the concept of conditional probability and provide an example..
 Take a deep breath, think step by step, and give an accurate response

Answer:  What is the difference between conditional probabilities (P/E) and conditional probability?
 P = P-1 / P - 1
 E = E-2 / E + 1

What are the differences in conditional probabilities from P to E?
 P = P-3 / P - 3
 E = E-4 / E + 4

How do I calculate conditional probabilities for a stock price? How can I determine if it's overvalued or undervaluing something? Is there
****************************************************************************************************
</s>Question: Detail the steps involved in finding the solution to a linear equation with three variables..
 Take a deep breath, think step by step, and give an accurate response

Answer:  What is the difference between a linear equation (linear equations) and a linear equation?
 A linear equation can be defined as a series of numbers that are not linear.
 A lin